In [2]:
cd Background

/home/kunjithapathams/Desktop/Data Analytics/OCR/Background


In [3]:
import random
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os
from PIL import Image
from skimage import io
from skimage import transform as tf
import scipy.misc
import numpy as np
import cv2


In [4]:
#blendImage('A.png')
#print('image created')

image created


In [14]:
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
fntSizes = [20,21,22,23,24,25,26,27,28,29]
filename = 'defaultBG.png'
def createsamples(text_pos,alpha,fnt,filename,seq):
    bgimage = Image.open(filename)
    draw  = ImageDraw.Draw(bgimage)
    draw.text(text_pos,alpha,font=fnt)
    bgimage.save('samples/'+alpha+'_'+str(seq)+'.png')
        
for alpha in alphanum:
    seq = 1
    for fntSize in fntSizes:
        for i in range (1,50):
            if alpha in ['A','B','C','D','E','F','G','H','J','K','L','N','O','P','Q','R','S','T','U','V','X','Y','Z']:
                if fntSize == 20:
                    text_Pos = (15,15)                
                elif fntSize == 21:
                    text_Pos = (15,14)
                elif fntSize == 22:
                    text_Pos = (15,13)
                elif fntSize == 23:
                    text_Pos = (15,12)
                elif fntSize == 24:
                    text_Pos = (14,11)
                elif fntSize == 25:
                    text_Pos = (14,10)
                elif fntSize == 28:
                    text_Pos = (13,9)    
                elif fntSize == 29:
                    text_Pos = (12,9)    
                elif fntSize in [26,27]:
                    text_Pos = (13,9)
            
            elif alpha in ['M','W']:
                if fntSize == 20:
                    text_Pos = (16,15)                
                elif fntSize == 21:
                    text_Pos = (16,14)
                elif fntSize == 22:
                    text_Pos = (16,13)
                elif fntSize == 23:
                    text_Pos = (16,12)
                elif fntSize == 24:
                    text_Pos = (15,11)
                elif fntSize == 25:
                    text_Pos = (15,10)
                elif fntSize == 28:
                    text_Pos = (14,9)    
                elif fntSize == 29:
                    text_Pos = (13,9)    
                elif fntSize in [26,27]:
                    text_Pos = (14,9)
            elif alpha == 'I':
                if fntSize == 20:
                    text_Pos = (21,15)                
                elif fntSize == 21:
                    text_Pos = (21,14)
                elif fntSize == 22:
                    text_Pos = (21,13)
                elif fntSize == 23:
                    text_Pos = (21,12)
                elif fntSize == 24:
                    text_Pos = (21,11)
                elif fntSize == 25:
                    text_Pos = (20,10)
                elif fntSize in [26,27,28,29]:
                    text_Pos = (20,9)
                
            elif alpha in ['0','1','2','3','4','5','6','7','8','9']:
                if fntSize == 20:
                    text_Pos = (17,15)                
                elif fntSize == 21:
                    text_Pos = (17,14)
                elif fntSize == 22:
                    text_Pos = (17,13)
                elif fntSize == 23:
                    text_Pos = (17,12)
                elif fntSize == 24:
                    text_Pos = (17,11)
                elif fntSize == 25:
                    text_Pos = (16,10)
                elif fntSize in [26,27,28,29]:
                    text_Pos = (16,9)
                
            fnt = ImageFont.truetype('arialbd.ttf', fntSize)
            createsamples(text_Pos,alpha,fnt,filename,seq)
            seq = seq +1
                
print ('samples created....')
           


In [7]:
def blendImage(inputFilename,outputFilename):
    

    #== Parameters =======================================================================
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (1.0,1.0,1.0) # In BGR format


    #== Processing =======================================================================

    #-- Read image -----------------------------------------------------------------------
    #print('samples/'+filename)
    img = cv2.imread(inputFilename)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    _,contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    #_, contours, _= cv2.findContours(skin_ycrcb, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]

    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))
    

    #-- Smooth mask, then blur it --------------------------------------------------------
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

    #-- Blend masked img into MASK_COLOR background --------------------------------------
    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
    img         = img.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 

    #cv2.imshow('img', masked)                                   # Display
    #cv2.waitKey()

    cv2.imwrite(outputFilename, masked)           # Save

In [6]:
def noisy(image):
    row,col = image.shape
    gauss = np.random.randn(row,col)
    gauss = gauss.reshape(row,col)        
    noisy = image + image * gauss
    return noisy

for filename in os.listdir('samples/'):        
    image = cv2.imread('samples/'+filename,0)
    image = cv2.blur(image,(5,5))
    cv2.imwrite('samplesBW/'+filename,image)
    #image = noisy(image)
    #print(filename)
    blendImage('samplesBW/'+filename,'samplesBW/'+filename)
    #cv2.imwrite('samplesBW/'+filename,image)

print('noice induced')



noice induced


In [8]:
print('start reshaping')
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
reshaped = np.empty([17640,2500])
Y = np.chararray([(len(alphanum))*490,1]) 
j=0
for alpha in alphanum:
    print(alpha)
    for i in range(1,490):
        filename = 'samplesBW/'+alpha+'_'+str(i)+'.png'
        image = io.imread(filename,'0')
        reshaped[j] = image.reshape(1,2500)
        #reshaped = np.concatenate([reshaped,image.reshape(1,2500)])
        Y[j] = alpha
        j = j+1

print('Reshaping Complete')
print('Starting to segregate Training and Validation sets')

from sklearn.model_selection import train_test_split
X_Train,X_Test,Y_Train,Y_Test = train_test_split(reshaped,Y)


print('Segregation Completed')

start reshaping
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
0
1
2
3
4
5
6
7
8
9
Reshaping Complete
Starting to segregate Training and Validation sets
Segregation Completed


In [9]:
print('starting to feature scaling')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_Train)
X_Train = scaler.transform(X_Train)
X_Test = scaler.transform(X_Test)

print('Feature Scaling Completed')
print('Start training')
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X_Train,Y_Train)


print('Training done...')
print('Now start predicting for the validation set')
predictions = mlp.predict(X_Test)

print('Predictions done')
print('evaluate the results')
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_Test,predictions))

starting to feature scaling
Feature Scaling Completed
Start training


/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training done...
Now start predicting for the validation set
Predictions done
evaluate the results
             precision    recall  f1-score   support

        b''       0.17      1.00      0.29         1
    b'\x07'       0.00      0.00      0.00         2
       b'0'       1.00      1.00      1.00       122
       b'1'       1.00      1.00      1.00       117
       b'2'       1.00      1.00      1.00       129
       b'3'       1.00      1.00      1.00       128
       b'4'       1.00      1.00      1.00       118
       b'5'       1.00      1.00      1.00       130
       b'6'       1.00      1.00      1.00       128
       b'7'       1.00      1.00      1.00       120
       b'8'       1.00      1.00      1.00       114
       b'9'       1.00      1.00      1.00       129
       b'A'       1.00      1.00      1.00       123
       b'B'       1.00      1.00      1.00       128
       b'C'       1.00      1.00      1.00       126
       b'D'       1.00      1.00      1.00       122

/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
for filename in os.listdir('Test/'):  
    BGImage = Image.open('defaultBG.png')
    img = Image.open('Test/'+filename)
    #offset = ((bg_w - img_w) / 2, (bg_h - img_h) / 2)
    BGImage.paste(img, (15,15))
    BGImage.save('TestBW/'+filename)
    blendImage('TestBW/'+filename,'TestBW/'+filename)
    
print('Test images created...')



Test images created...


In [33]:
print('starting to reshape the test images')
pannumber = ['A','W','S','P','K','5','9','6','7','E']
reshaped1 = np.empty([20,2500])
Y1=np.chararray([len(pannumber)*2,1])
j=0
for alpha in pannumber:
    for i in range (1,3):
        filename = 'TestBW/'+alpha+'_'+str(i)+'.PNG'
        print(filename)
        image = io.imread(filename,'0')    
        #print(image)
        reshaped1[j] = image.reshape(1,2500)
        Y1[j] = alpha
        j = j+1

print(reshaped1.shape)
print(Y1.shape)

print('reshaping done...')
print('start predicting')
X1_Test = scaler.transform(reshaped1)
predictions = mlp.predict(X1_Test)
print(classification_report(Y1,predictions))

starting to reshape the test images
TestBW/A_1.PNG
TestBW/A_2.PNG
TestBW/W_1.PNG
TestBW/W_2.PNG
TestBW/S_1.PNG
TestBW/S_2.PNG
TestBW/P_1.PNG
TestBW/P_2.PNG
TestBW/K_1.PNG
TestBW/K_2.PNG
TestBW/5_1.PNG
TestBW/5_2.PNG
TestBW/9_1.PNG
TestBW/9_2.PNG
TestBW/6_1.PNG
TestBW/6_2.PNG
TestBW/7_1.PNG
TestBW/7_2.PNG
TestBW/E_1.PNG
TestBW/E_2.PNG
(20, 2500)
(20, 1)
reshaping done...
start predicting
             precision    recall  f1-score   support

       b'2'       0.00      0.00      0.00         0
       b'5'       0.00      0.00      0.00         2
       b'6'       0.00      0.00      0.00         2
       b'7'       0.00      0.00      0.00         2
       b'9'       0.00      0.00      0.00         2
       b'A'       0.00      0.00      0.00         2
       b'E'       0.00      0.00      0.00         2
       b'K'       0.00      0.00      0.00         2
       b'M'       0.00      0.00      0.00         0
       b'N'       0.00      0.00      0.00         0
       b'P'       0.00    

/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [43]:
ct = 0
for character in Y_Test:
    #print(character)
    if character == '0':
        print(character)
        ct = ct +1
print(ct)

temp = ['A','B','C','D']
for character in temp:
    print(character)
    if character == 'A':
        print('found A')
        


0
A
found A
B
C
D
